Task 0

Preprocess dataset to split into 2 dataset files:

1 - time series

2,3 - 2024 data combined with socioeconomic param

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

# Load Data
df = pd.read_excel("data/cards/card_data.xlsx")
df = df[(df['Year'] >= 2012) & (df['Year'] <= 2024)].copy()

df.describe()

,Year,Postcode,GA,GA_Flag,HTA,HTA_Flag
count,41385.000000,41385.000000,41385.000000,13677.0,41385.000000,1415.0
mean,2017.991639,5114.405437,140.662018,1.0,819.617192,2.0
std,3.741829,2653.563728,339.230145,0.0,1582.307565,0.0
min,2012.000000,1000.000000,0.000000,1.0,0.000000,2.0
25%,2015.000000,2825.000000,10.389610,1.0,107.000000,2.0
50%,2018.000000,5046.000000,36.000000,1.0,272.000000,2.0
75%,2021.000000,7307.000000,114.000000,1.0,770.000000,2.0
max,2024.000000,9999.000000,4984.000000,1.0,22115.000000,2.0


In [2]:
# We create a subset of 2024 data for tasks 2 and 3 - clustering and socioeconomic analysis.

df_2024 = df[df['Year'] == 2024]
df_2024['postcode'] = df_2024.copy()['Postcode']
df_2024 = df_2024[['postcode', 'GA', 'HTA']].copy()

/var/folders/9c/8kt_75q92qv2p6wzt_p0h8tr0000gn/T/ipykernel_59923/601549539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['postcode'] = df_2024.copy()['Postcode']


In [3]:
# Load postcode GeoDataFrame
postcode_gdf = gpd.read_file("data/postcode/AMTOVZ_ZIP.shp")
postcode_gdf = postcode_gdf.to_crs("EPSG:4326")  # Ensure WGS84 (lat/lon)
postcode_gdf = postcode_gdf.rename(columns={'ZIP4': "postcode"})
postcode_gdf['area_km2'] = postcode_gdf.to_crs(epsg=3857).area / 1e6

# Use a spatial index to find neighbors
postcode_gdf['neighbors'] = None

for idx, row in postcode_gdf.iterrows():
    neighbors = postcode_gdf[postcode_gdf.geometry.touches(row.geometry)]
    postcode_gdf.at[idx, 'neighbors'] = list(neighbors['postcode'].values)

# Load GTFS stops and convert to GeoDataFrame
stops_df = pd.read_csv("data/gtfs/stops.txt")
stops_df = stops_df.dropna(subset=['stop_lat', 'stop_lon'])

stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat),
    crs="EPSG:4326"
)

# Spatial join: assign postcode zone to each stop based on location
stops_with_postcode = gpd.sjoin(stops_gdf, postcode_gdf[['postcode', 'geometry']], how="left", predicate="within")
stops_with_postcode = stops_with_postcode.dropna(subset=['postcode'])

routes = pd.read_csv("data/gtfs/routes.txt")
stop_times = pd.read_csv("data/gtfs/stop_times.txt")
stop_times['route_id'] = stop_times['trip_id'].str.extract(r'((?:\d+[-\w]*)+)', expand=False)
stop_frequency = stop_times.groupby('stop_id')['trip_id'].nunique().reset_index(name='daily_frequency')
stops_with_postcode = stops_with_postcode.merge(stop_frequency, on='stop_id', how='left').fillna(0)

postcode_stats = stops_with_postcode.groupby('postcode').agg(
    num_stops=('stop_id', 'nunique'),
    avg_daily_frequency=('daily_frequency', 'mean'),
    total_daily_frequency=('daily_frequency', 'sum'),
    max_stop_frequency=('daily_frequency', 'max'),
).reset_index()

postcode_stats = postcode_stats.merge(postcode_gdf[['postcode', 'area_km2', 'neighbors']], on='postcode', how='inner')
postcode_stats['stop_density'] = postcode_stats['num_stops'] / postcode_stats['area_km2']

def merge_unique_neighbors_exclude_self(series, current_postcode):
    merged = []
    for sublist in series:
        merged.extend(sublist)
    unique_neighbors = set(merged)
    if current_postcode in unique_neighbors:
        unique_neighbors.remove(current_postcode)
    return list(unique_neighbors)

def aggregate_neighbors(group):
    postcode = group['postcode'].iloc[0]
    neighbors = merge_unique_neighbors_exclude_self(group['neighbors'], postcode)
    return pd.Series({'num_stops': group['num_stops'].sum(),
                      'area_km2': group['area_km2'].sum(),
                      'avg_daily_frequency': group['avg_daily_frequency'].mean(),
                      'total_daily_frequency': group['total_daily_frequency'].sum(),
                      'max_stop_frequency': group['max_stop_frequency'].max(),
                      'neighbors': neighbors})

postcode_stats = postcode_stats.groupby('postcode').apply(aggregate_neighbors).reset_index()

def assign_region(postcode):
    if not str(postcode).isdigit():
        return "Unknown"
    pc = int(str(postcode).ljust(4, "0")[:4])  # Normalize to 4-digit
    if 1000 <= pc < 2000:
        return "Southwest"
    elif 2000 <= pc < 3000:
        return "Northwest"
    elif 3000 <= pc < 4000:
        return "Bern"
    elif 4000 <= pc < 5000:
        return "Basel"
    elif 5000 <= pc < 6000:
        return "Aarau"
    elif 6000 <= pc < 7000:
        return "Central"
    elif 7000 <= pc < 8000:
        return "Graubünden"
    elif 8000 <= pc < 9000:
        return "Zürich"
    elif 9000 <= pc < 10000:
        return "East"
    else:
        return "Other"

postcode_stats['region'] = postcode_stats['postcode'].apply(assign_region)
postcode_stats['postcode'] = postcode_stats['postcode'].map(lambda x: int(x))

/var/folders/9c/8kt_75q92qv2p6wzt_p0h8tr0000gn/T/ipykernel_59923/2728626.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  postcode_stats = postcode_stats.groupby('postcode').apply(aggregate_neighbors).reset_index()


In [4]:
postcode_stats

,postcode,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region
0,1000,108,29.157216,275.305556,29733.0,1347.0,"[1010, 1055, 1073, 1066, 1052, 1053, 1081, 1068]",Southwest
1,1003,38,1.705730,1110.078947,42183.0,8132.0,"[1007, 1005, 1006, 1004]",Southwest
2,1004,56,4.779186,688.732143,38569.0,3344.0,"[1018, 1008, 1003, 1007, 1005]",Southwest
3,1005,36,3.340990,826.277778,29746.0,7860.0,"[1010, 1009, 1018, 1012, 1003, 1011, 1006, 1004]",Southwest
4,1006,43,14.187561,542.348837,23321.0,3249.0,"[1007, 1003, 1005, 1009]",Southwest
...,...,...,...,...,...,...,...,...
3133,9652,2,7.252327,35.500000,71.0,71.0,"[9650, 9651, 9643]",East
3134,9655,6,26.518385,37.000000,222.0,74.0,"[9657, 9651, 8873, 9650, 9656]",East
3135,9656,10,71.961110,425.000000,4250.0,2014.0,"[9655, 8878, 9657, 8873, 8881]",East
3136,9657,14,42.709857,48.928571,685.0,212.0,"[9658, 9655, 8880, 9651, 8881, 9656, 9472]",East


In [5]:
merged_df = pd.merge(df_2024[['postcode','GA', 'HTA']], postcode_stats, on='postcode', how='left')
merged_df['stop_density'] = merged_df['num_stops'] / merged_df['area_km2']
merged_df

,postcode,GA,HTA,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region,stop_density
0,1000,95.000000,1674.0,108.0,29.157216,275.305556,29733.0,1347.0,"[1010, 1055, 1073, 1066, 1052, 1053, 1081, 1068]",Southwest,3.704057
1,1003,710.000000,4034.0,38.0,1.705730,1110.078947,42183.0,8132.0,"[1007, 1005, 1006, 1004]",Southwest,22.277846
2,1004,1642.000000,12892.0,56.0,4.779186,688.732143,38569.0,3344.0,"[1018, 1008, 1003, 1007, 1005]",Southwest,11.717476
3,1005,804.000000,6368.0,36.0,3.340990,826.277778,29746.0,7860.0,"[1010, 1009, 1018, 1012, 1003, 1011, 1006, 1004]",Southwest,10.775250
4,1006,1304.000000,8117.0,43.0,14.187561,542.348837,23321.0,3249.0,"[1007, 1003, 1005, 1009]",Southwest,3.030824
...,...,...,...,...,...,...,...,...,...,...,...
3170,9652,65.000000,336.0,2.0,7.252327,35.500000,71.0,71.0,"[9650, 9651, 9643]",East,0.275774
3171,9655,21.000000,112.0,6.0,26.518385,37.000000,222.0,74.0,"[9657, 9651, 8873, 9650, 9656]",East,0.226258
3172,9656,11.711111,219.0,10.0,71.961110,425.000000,4250.0,2014.0,"[9655, 8878, 9657, 8873, 8881]",East,0.138964
3173,9657,35.000000,276.0,14.0,42.709857,48.928571,685.0,212.0,"[9658, 9655, 8880, 9651, 8881, 9656, 9472]",East,0.327793


In [6]:
plz_map = pd.read_csv('data/raw_data/AMTOVZ_CSV_WGS84.csv')

# Group by 'PLZ'
plz_map['postcode'] = plz_map['PLZ']
grouped_plz = plz_map.groupby('postcode').agg(
    lon=('E', 'mean'),
    lat=('N', 'mean'),
    language=('Sprache', lambda x: x.mode()[0] if not x.mode().empty else None),
    canton=('Kantonskürzel', lambda x: x.mode()[0] if not x.mode().empty else None)
)

merged_df = pd.merge(merged_df, grouped_plz, on='postcode', how='inner')
merged_df


,postcode,GA,HTA,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region,stop_density,lon,lat,language,canton
0,1000,95.000000,1674.0,108.0,29.157216,275.305556,29733.0,1347.0,"[1010, 1055, 1073, 1066, 1052, 1053, 1081, 1068]",Southwest,3.704057,6.689084,46.556266,fr,VD
1,1003,710.000000,4034.0,38.0,1.705730,1110.078947,42183.0,8132.0,"[1007, 1005, 1006, 1004]",Southwest,22.277846,6.630105,46.520004,fr,VD
2,1004,1642.000000,12892.0,56.0,4.779186,688.732143,38569.0,3344.0,"[1018, 1008, 1003, 1007, 1005]",Southwest,11.717476,6.618678,46.528480,fr,VD
3,1005,804.000000,6368.0,36.0,3.340990,826.277778,29746.0,7860.0,"[1010, 1009, 1018, 1012, 1003, 1011, 1006, 1004]",Southwest,10.775250,6.642500,46.519859,fr,VD
4,1006,1304.000000,8117.0,43.0,14.187561,542.348837,23321.0,3249.0,"[1007, 1003, 1005, 1009]",Southwest,3.030824,6.637105,46.511217,fr,VD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170,9652,65.000000,336.0,2.0,7.252327,35.500000,71.0,71.0,"[9650, 9651, 9643]",East,0.275774,9.191749,47.238372,de,SG
3171,9655,21.000000,112.0,6.0,26.518385,37.000000,222.0,74.0,"[9657, 9651, 8873, 9650, 9656]",East,0.226258,9.228587,47.199399,de,SG
3172,9656,11.711111,219.0,10.0,71.961110,425.000000,4250.0,2014.0,"[9655, 8878, 9657, 8873, 8881]",East,0.138964,9.262987,47.183143,de,SG
3173,9657,35.000000,276.0,14.0,42.709857,48.928571,685.0,212.0,"[9658, 9655, 8880, 9651, 8881, 9656, 9472]",East,0.327793,9.298697,47.209321,de,SG


In [7]:
population_metrics = pd.read_excel("data/cleaned/population_metrics.xlsx")
merged_df = pd.merge(merged_df, population_metrics, on='postcode', how='inner')
merged_df

,postcode,GA,HTA,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region,...,language,canton,population,male_population,swiss_citizen,married,age_20_64,total_residential_housing,pure_residential_single,pure_residential_multiple
0,1000,95.000000,1674.0,108.0,29.157216,275.305556,29733.0,1347.0,"[1010, 1055, 1073, 1066, 1052, 1053, 1081, 1068]",Southwest,...,fr,VD,4332,2168,2496,1372,2764,755.0,504,193
1,1003,710.000000,4034.0,38.0,1.705730,1110.078947,42183.0,8132.0,"[1007, 1005, 1006, 1004]",Southwest,...,fr,VD,6762,3492,3503,1621,5181,525.0,7,67
2,1004,1642.000000,12892.0,56.0,4.779186,688.732143,38569.0,3344.0,"[1018, 1008, 1003, 1007, 1005]",Southwest,...,fr,VD,31611,15414,18111,8834,21757,1504.0,128,857
3,1005,804.000000,6368.0,36.0,3.340990,826.277778,29746.0,7860.0,"[1010, 1009, 1018, 1012, 1003, 1011, 1006, 1004]",Southwest,...,fr,VD,12416,6010,7350,3329,8641,829.0,116,395
4,1006,1304.000000,8117.0,43.0,14.187561,542.348837,23321.0,3249.0,"[1007, 1003, 1005, 1009]",Southwest,...,fr,VD,15519,7381,9467,4460,10323,813.0,52,430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,9652,65.000000,336.0,2.0,7.252327,35.500000,71.0,71.0,"[9650, 9651, 9643]",East,...,de,SG,750,380,628,319,423,167.0,130,51
3170,9655,21.000000,112.0,6.0,26.518385,37.000000,222.0,74.0,"[9657, 9651, 8873, 9650, 9656]",East,...,de,SG,342,178,318,148,192,368.0,83,34
3171,9656,11.711111,219.0,10.0,71.961110,425.000000,4250.0,2014.0,"[9655, 8878, 9657, 8873, 8881]",East,...,de,SG,671,336,562,270,352,456.0,184,80
3172,9657,35.000000,276.0,14.0,42.709857,48.928571,685.0,212.0,"[9658, 9655, 8880, 9651, 8881, 9656, 9472]",East,...,de,SG,737,376,625,314,402,1110.0,294,87


In [8]:
canton_economic = pd.read_excel('data/cleaned/canton_economic.xlsx')

# Merge the two DataFrames based on the 'canton' column
merged_df = pd.merge(merged_df, canton_economic, on='canton', how='left')

In [9]:
merged_df['GA_pct'] = merged_df['GA'] / merged_df['population'] * 100
merged_df['HTA_pct'] = merged_df['HTA'] / merged_df['population'] * 100
merged_df['GA_log'] = np.log(merged_df['GA'])
merged_df['HTA_log'] = np.log(merged_df['HTA'])

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
merged_df.to_csv('data/cleaned/ga_hta_socioeconomic_2024.csv', index=False)

In [11]:
# remove pii redacted data
print('original', len(merged_df))
merged_df = merged_df[merged_df['GA'] > 20]
merged_df = merged_df[merged_df['HTA'] > 20]
print('excluded to', len(merged_df))
merged_df.to_csv('data/cleaned/ga_hta_socioeconomic_2024_clean.csv', index=False)


original 3174
excluded to 2010


In [12]:
merged_df

,postcode,GA,HTA,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region,...,total_residential_housing,pure_residential_single,pure_residential_multiple,gdp_per_capita,eco_activity_rate,unemploy_rate,GA_pct,HTA_pct,GA_log,HTA_log
0,1000,95.0,1674.0,108.0,29.157216,275.305556,29733.0,1347.0,"[1010, 1055, 1073, 1066, 1052, 1053, 1081, 1068]",Southwest,...,755.0,504,193,78021,62.661725,7.273163,2.192982,38.642659,4.553877,7.422971
1,1003,710.0,4034.0,38.0,1.705730,1110.078947,42183.0,8132.0,"[1007, 1005, 1006, 1004]",Southwest,...,525.0,7,67,78021,62.661725,7.273163,10.499852,59.656906,6.565265,8.302514
2,1004,1642.0,12892.0,56.0,4.779186,688.732143,38569.0,3344.0,"[1018, 1008, 1003, 1007, 1005]",Southwest,...,1504.0,128,857,78021,62.661725,7.273163,5.194394,40.783272,7.403670,9.464362
3,1005,804.0,6368.0,36.0,3.340990,826.277778,29746.0,7860.0,"[1010, 1009, 1018, 1012, 1003, 1011, 1006, 1004]",Southwest,...,829.0,116,395,78021,62.661725,7.273163,6.475515,51.288660,6.689599,8.759041
4,1006,1304.0,8117.0,43.0,14.187561,542.348837,23321.0,3249.0,"[1007, 1003, 1005, 1009]",Southwest,...,813.0,52,430,78021,62.661725,7.273163,8.402603,52.303628,7.173192,9.001716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3167,9650,119.0,720.0,10.0,77.831042,225.900000,2259.0,1954.0,"[9655, 9651, 9643, 8873, 9642, 8723, 9652]",East,...,318.0,390,118,85320,65.426420,3.548117,6.633222,40.133779,4.779123,6.579251
3169,9652,65.0,336.0,2.0,7.252327,35.500000,71.0,71.0,"[9650, 9651, 9643]",East,...,167.0,130,51,85320,65.426420,3.548117,8.666667,44.800000,4.174387,5.817111
3170,9655,21.0,112.0,6.0,26.518385,37.000000,222.0,74.0,"[9657, 9651, 8873, 9650, 9656]",East,...,368.0,83,34,85320,65.426420,3.548117,6.140351,32.748538,3.044522,4.718499
3172,9657,35.0,276.0,14.0,42.709857,48.928571,685.0,212.0,"[9658, 9655, 8880, 9651, 8881, 9656, 9472]",East,...,1110.0,294,87,85320,65.426420,3.548117,4.748982,37.449118,3.555348,5.620401


Handle Task 1 - time series data

In [13]:
df = df.drop(columns=['GA_Flag', 'HTA_Flag'])

pivot_df = df.pivot_table(index='Postcode', columns=['Year'], values=['GA', 'HTA'])
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]
pivot_df = pivot_df.reset_index()

melted_ga = pivot_df[['Postcode'] + [col for col in pivot_df.columns if col.startswith('GA_')]].copy()
melted_ga['type'] = 'ga'
melted_hta = pivot_df[['Postcode'] + [col for col in pivot_df.columns if col.startswith('HTA_')]].copy()
melted_hta['type'] = 'hta'

ga_cols = ['Postcode'] + [col.replace('GA_', '') for col in melted_ga.columns if col.startswith('GA_')] + ['type']
melted_ga.columns = ga_cols
hta_cols = ['Postcode'] + [col.replace('HTA_', '') for col in melted_hta.columns if col.startswith('HTA_')] + ['type']
melted_hta.columns = hta_cols

result_df = pd.concat([melted_ga, melted_hta], ignore_index=True)
result_df = result_df.sort_values(by=['Postcode', 'type']).reset_index(drop=True)
result_df.dropna()
result_df = result_df[result_df['Postcode'] != 9999] # missing data as 9999 seems the postcode no longer exist

result_df['postcode'] = result_df['Postcode']
result_df = pd.merge(result_df, merged_df[['postcode', 'region']], on='postcode', how='left')
result_df.drop('Postcode', axis=1, inplace=True)
result_df.to_csv("data/cleaned/ga_hta_ts.csv", index=False)